In [3]:
import pandas as pd
import nltk
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
import string
# from nltk.stem import WordNetLemmatizer
#from stop_words import safe_get_stop_words
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch
#from transformers import BertTokenizer
#from torch.utils.data import Dataset, DataLoader
from torchtext import data
from torch.utils.data import DataLoader, TensorDataset

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [37]:
df=pd.read_csv("/content/drive/MyDrive/hindi_train_val.csv")

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20183 entries, 0 to 20182
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   20183 non-null  int64 
 1   text    20183 non-null  object
dtypes: int64(1), object(1)
memory usage: 315.5+ KB


In [39]:
text=df['text']
label=df['label']

In [40]:
df2=pd.concat([text,label],axis=1)

In [41]:
df2.head()

text  label
0              भीड़ में  बहुत  हब्सी मिलेंगे      0
1  साले बेवकूफ अपनी मां मक्खियां  तो हटा दें      0
2           बुर देदो तो मुह में लंड ले लो तो      0
3       कुत्ता वहा है चिल्ला तू क्यों रहा है      0
4  चाय नहीं पीता हूं मैं इसी को छोड़ दिया ok      1

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [42]:
sw=["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप","इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी","इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके","उनको","उन्हीं","उन्हें","उन्हों","उस","उसके","उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर","और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा","का","काफि","काफ़ी","कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर","किस","किसि","किसी","किसे","की","कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ","जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे","जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि","थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे","निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला","बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि","यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग","वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि","सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है","हैं","हो","होता","होति","होती","होते","होना","होने"]
stop_words = set(sw)
stop_word_parameter=list(sw)

In [43]:
import re
emoji_pattern = re.emoj = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)


In [44]:
for i in range(len(df2)):
    df2['text'][i] = emoji_pattern.sub(r'', df2['text'][i])

<ipython-input-44-c213bb15eb89>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['text'][i] = emoji_pattern.sub(r'', df2['text'][i])


In [45]:
df2['text'] =  [re.sub(r'[?|$|.|!|।|,|:|;|]','', str(x)) for x in df2['text']]

In [46]:
df2['text'] = df2['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (sw)]))

In [47]:
df3=df2.copy()
df4=df2.copy()
texts=df2['text'].to_list()


In [48]:
def hindi_tokenizer(text):
    """
    Tokenizes a Hindi sentence based on whitespace.
    """
    tokens = text.strip().split()
    return tokens


# Tokenize the sentences in the DataFrame
df3['tokens'] = df3['text'].apply(hindi_tokenizer)

In [16]:
text2 = ' '.join(texts)
words=text2.strip().split()

In [17]:
from collections import Counter
count_words = Counter(words)
total_words = len(words)
sorted_words = count_words.most_common(total_words)
print(f'count_words={count_words},total_words={total_words},sorted_words={sorted_words}')

count_words=Counter({'क्या': 2378, 'जी': 1388, 'नही': 1285, 'वीडियो': 1195, 'तुम': 1009, 'रही': 991, 'सब': 960, 'जय': 890, 'बात': 877, 'लोग': 861, 'हम': 849, 'करो': 835, 'अब': 793, 'हर': 758, 'वो': 752, 'तेरी': 734, 'चाहिए': 710, 'तू': 704, 'दे': 687, 'मैं': 685, 'दिल': 685, 'देश': 653, 'पानी': 646, 'शर्म': 640, 'मेरे': 633, 'मुझे': 633, 'मां': 627, 'साले': 623, 'भाई': 601, 'प्यार': 554, 'बहन': 545, 'राम': 541, 'मेरी': 539, 'कभी': 537, 'तेरे': 515, 'आ': 502, 'ले': 490, 'मत': 486, 'नाम': 484, 'अच्छा': 480, 'अगर': 479, 'होगा': 465, 'ऐसा': 462, 'पागल': 451, 'मोदी': 444, 'आज': 444, 'क्यों': 442, 'बना': 441, 'कुत्ते': 441, 'काम': 427, 'देख': 413, 'गई': 412, 'हूं': 393, 'श्री': 392, 'बाप': 387, 'माँ': 376, 'मेरा': 376, 'ज्यादा': 371, 'लो': 369, 'साला': 368, 'अरे': 367, 'जाता': 366, 'पता': 363, 'पास': 356, 'बार': 355, 'कपड़े': 354, 'कैसे': 352, 'यार': 352, 'सुंदर': 351, 'लग': 349, 'वाला': 342, 'गांड': 341, 'लोगों': 339, 'दिन': 337, 'लड़की': 334, 'आपको': 331, 'आपकी': 327, 'इतना': 325, 'रंडी': 

In [20]:
vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

In [21]:
reviews_int = []
for review in texts:
    r = [vocab_to_int[w] for w in review.split()]
    reviews_int.append(r)

In [22]:
encoded_labels = [1 if label ==1 else 0 for label in df3['label']]
encoded_labels = np.array(encoded_labels)

reviews_len = [len(x) for x in reviews_int]
reviews_int = [ reviews_int[i] for i, l in enumerate(reviews_len) if l>0 ]
encoded_labels = [ encoded_labels[i] for i, l in enumerate(reviews_len) if l> 0 ]

In [49]:
from sklearn.model_selection import train_test_split

X =df2['text'] # your input features
y =df2['label'] # your target variable

# Split the data into a training set and a validation set
X_train_text, X_val_text, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)



In [24]:

def pad_features(reviews_int, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's or truncated to the input seq_length.
    '''
    features = np.zeros((len(reviews_int), seq_length), dtype = int)

    
    for i, review in enumerate(reviews_int):
        review_len = len(review)
        
        if review_len <= seq_length:
            zeroes = list(np.zeros(seq_length-review_len))
            new = zeroes+review
        elif review_len > seq_length:
            new = review[0:seq_length]
        
        features[i,:] = np.array(new)
    

    print(len(features))  
    return features
features=pad_features(reviews_int,1)

20175


In [25]:
split_frac = 0.6
len_feat=len(features)
train_x = features[0:int(split_frac*len_feat)]
train_y = encoded_labels[0:int(split_frac*len_feat)]
remaining_x = features[int(split_frac*len_feat):]
remaining_y = encoded_labels[int(split_frac*len_feat):]
valid_x = remaining_x[0:int(len(remaining_x)*0.5)]
valid_y = remaining_y[0:int(len(remaining_y)*0.5)]
test_x = remaining_x[int(len(remaining_x)*0.5):]
test_y = remaining_y[int(len(remaining_y)*0.5):]
train_y=np.asarray(train_y)
test_y=np.asarray(test_y)
valid_y=np.asarray(valid_y)
train_x

array([[1431],
       [  28],
       [1039],
       ...,
       [1854],
       [ 319],
       [1230]])

In [26]:
# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(valid_x), torch.from_numpy(valid_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))
# dataloaders
batch_size = 50
# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size,drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size,drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size,drop_last=True)

In [27]:
dataiter = iter(train_loader)
sample_x, sample_y =next(dataiter)

In [28]:
class SentimentLSTM(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super().__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

In [29]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2
net = SentimentLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
print(net)

SentimentLSTM(
  (embedding): Embedding(30891, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [30]:
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


# training params

epochs = 15 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping
train_on_gpu=1
# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        inputs = inputs.type(torch.LongTensor)
        output, h = net(inputs.to('cuda'), h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                inputs = inputs.type(torch.LongTensor)
                output, val_h = net(inputs.to('cuda'), val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/15... Step: 100... Loss: 0.623887... Val Loss: 0.625400
Epoch: 1/15... Step: 200... Loss: 0.555387... Val Loss: 0.609194
Epoch: 2/15... Step: 300... Loss: 0.539499... Val Loss: 0.606080
Epoch: 2/15... Step: 400... Loss: 0.627375... Val Loss: 0.618765
Epoch: 3/15... Step: 500... Loss: 0.528284... Val Loss: 0.612797
Epoch: 3/15... Step: 600... Loss: 0.552499... Val Loss: 0.637880
Epoch: 3/15... Step: 700... Loss: 0.533693... Val Loss: 0.628193
Epoch: 4/15... Step: 800... Loss: 0.329122... Val Loss: 0.657776
Epoch: 4/15... Step: 900... Loss: 0.480945... Val Loss: 0.653635
Epoch: 5/15... Step: 1000... Loss: 0.296142... Val Loss: 0.687404
Epoch: 5/15... Step: 1100... Loss: 0.472398... Val Loss: 0.672118
Epoch: 5/15... Step: 1200... Loss: 0.468136... Val Loss: 0.688962
Epoch: 6/15... Step: 1300... Loss: 0.450103... Val Loss: 0.748970
Epoch: 6/15... Step: 1400... Loss: 0.425934... Val Loss: 0.740229
Epoch: 7/15... Step: 1500... Loss: 0.308039... Val Loss: 0.742365
Epoch: 7/15... Step

In [31]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    inputs = inputs.type(torch.LongTensor)
    output, h = net(inputs.to('cuda'), h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.7f}".format(test_acc))

Test loss: 0.874
Test accuracy: 0.6733581


FOR TESTING(UPLOAD THE TRAIN AND TEST FILE IN DRIVE HOME FOLDER AND CHANGE FILENAME ACCORDINGLY)

In [53]:
df=pd.read_csv("/content/drive/MyDrive/hindi_test.csv")

In [54]:
text=df['text']
label=df['label']

In [55]:
df2=pd.concat([text,label],axis=1)

In [8]:
df2.head()

text  label
0  मैं ये नही सोच रहा की इसे निकले कैसे मैं ये सो...      0
1        और दिवाली में भी पूरा देश पड़ाका नहीं फोडात      1
2      कुत्ता बिल्ली पाल लेना मगर गलत फहमी कभी नहीं।      1
3      तेरी गांड में प्याज काट देगा गुज्जर भोसड़ी के      0
4            बंगाली साड़ी ऐसे नहीं पहना जाता है दीदी      1

In [56]:
sw=["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप","इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी","इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके","उनको","उन्हीं","उन्हें","उन्हों","उस","उसके","उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर","और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा","का","काफि","काफ़ी","कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर","किस","किसि","किसी","किसे","की","कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ","जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे","जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि","थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे","निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला","बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि","यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग","वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि","सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है","हैं","हो","होता","होति","होती","होते","होना","होने"]
stop_words = set(sw)
stop_word_parameter=list(sw)

In [57]:
import re
emoji_pattern = re.emoj = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)


In [58]:
for i in range(len(df2)):
    df2['text'][i] = emoji_pattern.sub(r'', df2['text'][i])

<ipython-input-58-c213bb15eb89>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['text'][i] = emoji_pattern.sub(r'', df2['text'][i])


In [59]:
df2['text'] =  [re.sub(r'[?|$|.|!|।|,|:|;|]','', str(x)) for x in df2['text']]

In [60]:
df2['text'] = df2['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (sw)]))

In [61]:
df3=df2.copy()
df4=df2.copy()
texts=df2['text'].to_list()


In [62]:
def hindi_tokenizer(text):
    """
    Tokenizes a Hindi sentence based on whitespace.
    """
    tokens = text.strip().split()
    return tokens


# Tokenize the sentences in the DataFrame
df3['tokens'] = df3['text'].apply(hindi_tokenizer)

In [16]:
text2 = ' '.join(texts)
words=text2.strip().split()

In [17]:
from collections import Counter
count_words = Counter(words)
total_words = len(words)
sorted_words = count_words.most_common(total_words)
print(f'count_words={count_words},total_words={total_words},sorted_words={sorted_words}')

count_words=Counter({'क्या': 786, 'नही': 469, 'जी': 448, 'वीडियो': 367, 'तुम': 325, 'सब': 323, 'बात': 319, 'हम': 291, 'रही': 286, 'अब': 284, 'करो': 279, 'हर': 272, 'लोग': 272, 'वो': 271, 'भाई': 269, 'जय': 266, 'चाहिए': 260, 'मैं': 259, 'दे': 250, 'मेरे': 237, 'दिल': 233, 'तेरी': 230, 'तू': 225, 'देश': 221, 'शर्म': 216, 'बहन': 209, 'पानी': 208, 'मुझे': 203, 'हूं': 200, 'आ': 196, 'ले': 196, 'मां': 189, 'मोदी': 189, 'साले': 188, 'होगा': 185, 'प्यार': 182, 'अगर': 180, 'कभी': 178, 'मत': 177, 'काम': 175, 'तेरे': 175, 'नाम': 174, 'मेरी': 170, 'क्यों': 159, 'अच्छा': 155, 'मेरा': 152, 'गई': 152, 'कपड़े': 152, 'आज': 151, 'जाता': 147, 'बाप': 145, 'बना': 144, 'देख': 144, 'पागल': 143, 'लो': 141, 'सुंदर': 138, 'कुत्ते': 135, 'लग': 135, 'यार': 132, 'गए': 130, 'ऐसा': 129, 'ज्यादा': 127, 'साला': 127, 'अरे': 125, 'आपको': 125, 'भारत': 122, 'बस': 118, 'बार': 117, 'पता': 117, 'लड़की': 116, 'श्री': 115, 'लगता': 115, 'दिन': 114, 'वाला': 112, 'सरकार': 111, 'माँ': 111, 'गुड': 110, 'इतना': 105, 'पास': 105, 'सही

In [18]:
vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

In [19]:
reviews_int = []
for review in texts:
    r = [vocab_to_int[w] for w in review.split()]
    reviews_int.append(r)

In [20]:
encoded_labels = [1 if label ==1 else 0 for label in df3['label']]
encoded_labels = np.array(encoded_labels)

reviews_len = [len(x) for x in reviews_int]
reviews_int = [ reviews_int[i] for i, l in enumerate(reviews_len) if l>0 ]
encoded_labels = [ encoded_labels[i] for i, l in enumerate(reviews_len) if l> 0 ]

In [63]:
from sklearn.model_selection import train_test_split

X =df2['text'] # your input features
y =df2['label'] # your target variable

# Split the data into a training set and a validation set
X_train_text_test, X_val_text_test, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)



In [22]:

def pad_features(reviews_int, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's or truncated to the input seq_length.
    '''
    features = np.zeros((len(reviews_int), seq_length), dtype = int)

    
    for i, review in enumerate(reviews_int):
        review_len = len(review)
        
        if review_len <= seq_length:
            zeroes = list(np.zeros(seq_length-review_len))
            new = zeroes+review
        elif review_len > seq_length:
            new = review[0:seq_length]
        
        features[i,:] = np.array(new)
    

    print(len(features))  
    return features
features=pad_features(reviews_int,1)

6724


In [27]:
split_frac = 0.6
len_feat=len(features)
train_x_test = features[0:int(split_frac*len_feat)]
train_y_test = encoded_labels[0:int(split_frac*len_feat)]
remaining_x_test = features[int(split_frac*len_feat):]
remaining_y_test = encoded_labels[int(split_frac*len_feat):]
valid_x_test = remaining_x_test[0:int(len(remaining_x_test)*0.5)]
valid_y_test = remaining_y_test[0:int(len(remaining_y_test)*0.5)]
test_x_test = remaining_x_test[int(len(remaining_x_test)*0.5):]
test_y_test = remaining_y_test[int(len(remaining_y_test)*0.5):]
train_y_test=np.asarray(train_y_test)
test_y_test=np.asarray(test_y_test)
valid_y_test=np.asarray(valid_y_test)

In [29]:
# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x_test), torch.from_numpy(train_y_test))
valid_data = TensorDataset(torch.from_numpy(valid_x_test), torch.from_numpy(valid_y_test))
test_data = TensorDataset(torch.from_numpy(test_x_test), torch.from_numpy(test_y_test))
# dataloaders
batch_size = 50
# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size,drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size,drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size,drop_last=True)

In [30]:
dataiter = iter(train_loader)
sample_x, sample_y =next(dataiter)

In [31]:
class SentimentLSTM(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super().__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

In [32]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2
net = SentimentLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
print(net)

SentimentLSTM(
  (embedding): Embedding(15074, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [34]:
lr=0.001
train_on_gpu=1
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


# training params

epochs = 15 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping
train_on_gpu=1
# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        inputs = inputs.type(torch.LongTensor)
        output, h = net(inputs.to('cuda'), h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                inputs = inputs.type(torch.LongTensor)
                output, val_h = net(inputs.to('cuda'), val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 2/15... Step: 100... Loss: 0.439318... Val Loss: 0.648148
Epoch: 3/15... Step: 200... Loss: 0.672046... Val Loss: 0.693140
Epoch: 4/15... Step: 300... Loss: 0.451041... Val Loss: 0.696208
Epoch: 5/15... Step: 400... Loss: 0.403832... Val Loss: 0.717130
Epoch: 7/15... Step: 500... Loss: 0.379066... Val Loss: 0.771063
Epoch: 8/15... Step: 600... Loss: 0.231282... Val Loss: 0.840438
Epoch: 9/15... Step: 700... Loss: 0.281239... Val Loss: 0.864775
Epoch: 10/15... Step: 800... Loss: 0.315980... Val Loss: 0.834960
Epoch: 12/15... Step: 900... Loss: 0.237376... Val Loss: 0.934889
Epoch: 13/15... Step: 1000... Loss: 0.309782... Val Loss: 0.914695
Epoch: 14/15... Step: 1100... Loss: 0.231843... Val Loss: 0.933564
Epoch: 15/15... Step: 1200... Loss: 0.278258... Val Loss: 0.939099


In [51]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    inputs = inputs.type(torch.LongTensor)
    output, h = net(inputs.to('cuda'), h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.7f}".format(test_acc))

Test loss: 0.969
Test accuracy: 0.6104089


In [65]:
intersect = np.intersect1d(X_train_text, X_val_text_test)
#print(intersect)
print(len(intersect))

15


In [64]:
intersect = np.intersect1d(X_val_text, X_val_text_test)
#print(intersect)
print(len(intersect))

4
